<html>
    <leftr><span style="font-size: 32px">Toy Machine Learning Pipeline</span><br><span style="font-size: 25px">Exploratory Data Analysis</span></center>
    <br>
</html>

*Last update: 11/02/2021*

The present notebook is a companion analysis of the current state of the pipeline. Summarizing the data cleaning, feature generation, and how the model was fit.

In the following notebooks there will be new feature exploration and model alternatives.

**How to run this notebook?**

```bash
# build the image

# start the container
docker run -d -p 8888:8888 --env-file=./.env --mount type=bind,src="$PWD"/eda,dst=/app/eda toy-ml-pipeline
```

In [19]:
# import modules
from utils.io import list_files, load_output_df
import matplotlib.pyplot as pl
import seaborn as sns

## 1. Load the data

How can we access to the data in buckets?

In this section we will used the function written in the module `utils.io` to access the data stored in s3 buckets.

Firstly, we can use the function `utils.io.list_files()` to explore into each component's buckets.

In [12]:
list_files('')

['toy-applied-ml-pipeline/demo',
 'toy-applied-ml-pipeline/dev',
 'toy-applied-ml-pipeline/prod',
 'toy-applied-ml-pipeline/scratch',
 'toy-applied-ml-pipeline/test']

In [13]:
list_files('dev')

['toy-applied-ml-pipeline/dev/',
 'toy-applied-ml-pipeline/dev/clean',
 'toy-applied-ml-pipeline/dev/clean_online',
 'toy-applied-ml-pipeline/dev/features',
 'toy-applied-ml-pipeline/dev/features_online',
 'toy-applied-ml-pipeline/dev/inference',
 'toy-applied-ml-pipeline/dev/predictions_online',
 'toy-applied-ml-pipeline/dev/test',
 'toy-applied-ml-pipeline/dev/test_online',
 'toy-applied-ml-pipeline/dev/train_online',
 'toy-applied-ml-pipeline/dev/training']

In [14]:
list_files('dev/clean')

['toy-applied-ml-pipeline/dev/clean/',
 'toy-applied-ml-pipeline/dev/clean/2020_01',
 'toy-applied-ml-pipeline/dev/clean/2020_02',
 'toy-applied-ml-pipeline/dev/clean/2020_03',
 'toy-applied-ml-pipeline/dev/clean/2020_04',
 'toy-applied-ml-pipeline/dev/clean/2020_05',
 'toy-applied-ml-pipeline/dev/clean/2020_06',
 'toy-applied-ml-pipeline/dev/clean/2020_07',
 'toy-applied-ml-pipeline/dev/clean/2020_08',
 'toy-applied-ml-pipeline/dev/clean/2020_09',
 'toy-applied-ml-pipeline/dev/clean/2020_10',
 'toy-applied-ml-pipeline/dev/clean/2020_11',
 'toy-applied-ml-pipeline/dev/clean/2020_12',
 'toy-applied-ml-pipeline/dev/clean/20210302-163649.pq',
 'toy-applied-ml-pipeline/dev/clean/test.pq']

In [15]:
list_files('dev/clean/2020_01')

['toy-applied-ml-pipeline/dev/clean/2020_01/20210307-172352.pq',
 'toy-applied-ml-pipeline/dev/clean/2020_01/20210414-174617.pq',
 'toy-applied-ml-pipeline/dev/clean/2020_01/20210419-130349.pq',
 'toy-applied-ml-pipeline/dev/clean/2020_01/20210421-155643.pq',
 'toy-applied-ml-pipeline/dev/clean/2020_01/20210429-192900.pq']

Secondly, the function `utils.io.load_data_df()`:

In [ ]:
df=load_output_df('clean', version='2020_01')

### 1.1 Load multiple periods

Iterate trough different versions (i.e. `2020_01`, `2020_02`), load, and bind the data into a single dataframe.

Understand the computation steps involved to clean the data performed by `etl/cleaning.py`.

## 2. Exploratory Data Analysis

### 2.1 Data structure

The *codebook* for this dataset is detailed in the [README.md](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

### 2.2 Feature engineering

In this section, we explore and reproduce the featured generated by `etl/featuregen.py`.

## 3. Model development

Train the current model...

### 3.1 Fit the current model

### 3.2 How the model wrapper works?